In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import seaborn as sns
from apikeys import apikeys

In [5]:
# Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
for result in results():
    scores = analyzer.polarity_scores(tracklist['lyrics'])
    compound = scores['compound']
    pos = scores['pos']
    neu = scores['neu']
    neg = scores['neg']

    # Add sentiments for each tweet into `sentiments` list
    sentiments.append({"Date": tweet["created_at"], 
                       "Compound": compound,
                       "Positive": pos,
                       "Negative": neu,
                       "Neutral": neg})

print(sentiments)